In [2]:
from collections import defaultdict, Counter
import json
import os
import re

from tqdm import tqdm
import pandas as pd

Public challenge data should be extracted to a directory called data.

In [40]:
# Create a directory for all the generated files during execution of notebook.
!mkdir output_data

A subdirectory or file output_data already exists.


## Data Preprocessing

In [41]:
char_mappings = {
    "٥": "5",
    "А": "a",
    "В": "b",
    "Е": "e",
    "Н": "h",
    "Р": "P",
    "С": "C",
    "Т": "T",
    "а": "a",
    "г": "r",
    "е": "e",
    "к": "k",
    "м": "m",
    "о": "o",
    "р": "p",
    "ڈ": "د",
    "ڇ": "چ",
    # Persian numbers (will be raplaced by english one)
    "۰": "0",
    "۱": "1",
    "۲": "2",
    "۳": "3",
    "۴": "4",
    "۵": "5",
    "۶": "6",
    "۷": "7",
    "۸": "8",
    "۹": "9",
    ".": ".",
    # Arabic numbers (will be raplaced by english one)
    "٠": "0",
    "١": "1",
    "٢": "2",
    "٣": "3",
    "٤": "4",
    "٥": "5",
    "٦": "6",
    "٧": "7",
    "٨": "8",
    "٩": "9",
    # Special Arabic Characters (will be replaced by persian one)
    "ك": "ک",
    "ى": "ی",
    "ي": "ی",
    "ؤ": "و",
    "ئ": "ی",
    "إ": "ا",
    "أ": "ا",
    "آ": "ا",
    "ة": "ه",
    "ء": "ی",
    # French alphabet (will be raplaced by english one)
    "à": "a",
    "ä": "a",
    "ç": "c",
    "é": "e",
    "è": "e",
    "ê": "e",
    "ë": "e",
    "î": "i",
    "ï": "i",
    "ô": "o",
    "ù": "u",
    "û": "u",
    "ü": "u",
    # Camma (will be replaced by dots for floating point numbers)
    ",": ".",
    # And (will be replaced by dots for floating point numbers)
    "&": " and ",
    # Vowels (will be removed)
    "ّ": "",  # tashdid
    "َ": "",  # a
    "ِ": "",  # e
    "ُ": "",  # o
    "ـ": "",  # tatvil
    # Spaces
    "‍": "",  # 0x9E -> ZERO WIDTH JOINER
    "‌": " ",  # 0x9D -> ZERO WIDTH NON-JOINER
    # Arabic Presentation Forms-A (will be replaced by persian one)
    "ﭐ": "ا",
    "ﭑ": "ا",
    "ﭖ": "پ",
    "ﭗ": "پ",
    "ﭘ": "پ",
    "ﭙ": "پ",
    "ﭞ": "ت",
    "ﭟ": "ت",
    "ﭠ": "ت",
    "ﭡ": "ت",
    "ﭺ": "چ",
    "ﭻ": "چ",
    "ﭼ": "چ",
    "ﭽ": "چ",
    "ﮊ": "ژ",
    "ﮋ": "ژ",
    "ﮎ": "ک",
    "ﮏ": "ک",
    "ﮐ": "ک",
    "ﮑ": "ک",
    "ﮒ": "گ",
    "ﮓ": "گ",
    "ﮔ": "گ",
    "ﮕ": "گ",
    "ﮤ": "ه",
    "ﮥ": "ه",
    "ﮦ": "ه",
    "ﮪ": "ه",
    "ﮫ": "ه",
    "ﮬ": "ه",
    "ﮭ": "ه",
    "ﮮ": "ی",
    "ﮯ": "ی",
    "ﮰ": "ی",
    "ﮱ": "ی",
    "ﯼ": "ی",
    "ﯽ": "ی",
    "ﯾ": "ی",
    "ﯿ": "ی",
    # Arabic Presentation Forms-B (will be removed)
    "ﹰ": "",
    "ﹱ": "",
    "ﹲ": "",
    "ﹳ": "",
    "ﹴ": "",
    "﹵": "",
    "ﹶ": "",
    "ﹷ": "",
    "ﹸ": "",
    "ﹹ": "",
    "ﹺ": "",
    "ﹻ": "",
    "ﹼ": "",
    "ﹽ": "",
    "ﹾ": "",
    "ﹿ": "",
    # Arabic Presentation Forms-B (will be replaced by persian one)
    "ﺀ": "ی",
    "ﺁ": "ا",
    "ﺂ": "ا",
    "ﺃ": "ا",
    "ﺄ": "ا",
    "ﺅ": "و",
    "ﺆ": "و",
    "ﺇ": "ا",
    "ﺈ": "ا",
    "ﺉ": "ی",
    "ﺊ": "ی",
    "ﺋ": "ی",
    "ﺌ": "ی",
    "ﺍ": "ا",
    "ﺎ": "ا",
    "ﺏ": "ب",
    "ﺐ": "ب",
    "ﺑ": "ب",
    "ﺒ": "ب",
    "ﺓ": "ه",
    "ﺔ": "ه",
    "ﺕ": "ت",
    "ﺖ": "ت",
    "ﺗ": "ت",
    "ﺘ": "ت",
    "ﺙ": "ث",
    "ﺚ": "ث",
    "ﺛ": "ث",
    "ﺜ": "ث",
    "ﺝ": "ج",
    "ﺞ": "ج",
    "ﺟ": "ج",
    "ﺠ": "ج",
    "ﺡ": "ح",
    "ﺢ": "ح",
    "ﺣ": "ح",
    "ﺤ": "ح",
    "ﺥ": "خ",
    "ﺦ": "خ",
    "ﺧ": "خ",
    "ﺨ": "خ",
    "ﺩ": "د",
    "ﺪ": "د",
    "ﺫ": "ذ",
    "ﺬ": "ذ",
    "ﺭ": "ر",
    "ﺮ": "ر",
    "ﺯ": "ز",
    "ﺰ": "ز",
    "ﺱ": "س",
    "ﺲ": "س",
    "ﺳ": "س",
    "ﺴ": "س",
    "ﺵ": "ش",
    "ﺶ": "ش",
    "ﺷ": "ش",
    "ﺸ": "ش",
    "ﺹ": "ص",
    "ﺺ": "ص",
    "ﺻ": "ص",
    "ﺼ": "ص",
    "ﺽ": "ض",
    "ﺾ": "ض",
    "ﺿ": "ض",
    "ﻀ": "ض",
    "ﻁ": "ط",
    "ﻂ": "ط",
    "ﻃ": "ط",
    "ﻄ": "ط",
    "ﻅ": "ظ",
    "ﻆ": "ظ",
    "ﻇ": "ظ",
    "ﻈ": "ظ",
    "ﻉ": "ع",
    "ﻊ": "ع",
    "ﻋ": "ع",
    "ﻌ": "ع",
    "ﻍ": "غ",
    "ﻎ": "غ",
    "ﻏ": "غ",
    "ﻐ": "غ",
    "ﻑ": "ف",
    "ﻒ": "ف",
    "ﻓ": "ف",
    "ﻔ": "ف",
    "ﻕ": "ق",
    "ﻖ": "ق",
    "ﻗ": "ق",
    "ﻘ": "ق",
    "ﻙ": "ک",
    "ﻚ": "ک",
    "ﻛ": "ک",
    "ﻜ": "ک",
    "ﻝ": "ل",
    "ﻞ": "ل",
    "ﻟ": "ل",
    "ﻠ": "ل",
    "ﻡ": "م",
    "ﻢ": "م",
    "ﻣ": "م",
    "ﻤ": "م",
    "ﻥ": "ن",
    "ﻦ": "ن",
    "ﻧ": "ن",
    "ﻨ": "ن",
    "ﻩ": "ه",
    "ﻪ": "ه",
    "ﻫ": "ه",
    "ﻬ": "ه",
    "ﻭ": "و",
    "ﻮ": "و",
    "ﻯ": "ی",
    "ﻰ": "ی",
    "ﻱ": "ی",
    "ﻲ": "ی",
    "ﻳ": "ی",
    "ﻴ": "ی",
    "ﻵ": "لا",
    "ﻶ": "لا",
    "ﻷ": "لا",
    "ﻸ": "لا",
    "ﻹ": "لا",
    "ﻺ": "لا",
    "ﻻ": "لا",
    "ﻼ": "لا",
}

valid_chars = [
    " ",
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9",
    "A",
    "B",
    "C",
    "D",
    "E",
    "F",
    "G",
    "H",
    "I",
    "J",
    "K",
    "L",
    "M",
    "N",
    "O",
    "P",
    "Q",
    "R",
    "S",
    "T",
    "U",
    "V",
    "W",
    "X",
    "Y",
    "Z",
    "a",
    "b",
    "c",
    "d",
    "e",
    "f",
    "g",
    "h",
    "i",
    "j",
    "k",
    "l",
    "m",
    "n",
    "o",
    "p",
    "q",
    "r",
    "s",
    "t",
    "u",
    "v",
    "w",
    "x",
    "y",
    "z",
    "ا",
    "ب",
    "ت",
    "ث",
    "ج",
    "ح",
    "خ",
    "د",
    "ذ",
    "ر",
    "ز",
    "س",
    "ش",
    "ص",
    "ض",
    "ط",
    "ظ",
    "ع",
    "غ",
    "ف",
    "ق",
    "ل",
    "م",
    "ن",
    "ه",
    "و",
    "پ",
    "چ",
    "ژ",
    "ک",
    "گ",
    "ی",
]

translation_table = dict((ord(a), b) for a, b in char_mappings.items())

# Create a regex for recognizing invalid characters.
nonvalid_reg_text = '[^{}]'.format("".join(valid_chars))
nonvalid_reg = re.compile(nonvalid_reg_text)


def normalize_text(text, to_lower=True, remove_invalid=True):
    # Map invalid characters with replacement to valid characters.
    text = text.translate(translation_table)
    if to_lower:
        text = text.lower()
    if remove_invalid:
        text = nonvalid_reg.sub(' ', text)
    # Replace consecutive whitespaces with a single space character.
    text = re.sub(r"\s+", " ", text)
    return text

In [42]:
def read_json_lines(path, n_lines=None):
    """Creates a generator which reads and returns lines of
    a json lines file, one line at a time, each as a dictionary.
    
    This could be used as a memory-efficient alternative of `pandas.read_json`
    for reading a json lines file.
    """
    with open(path, 'r') as f:
        for i, line in enumerate(f):
            if n_lines == i:
                break
            yield json.loads(line)

In [43]:
class JSONLinesWriter:
    """
    Helper class to write list of dictionaries into a file in json lines
    format, i.e. one json record per line.
    """

    def __init__(self, file_path):
        self.fd = None
        self.file_path = file_path
        self.delimiter = "\n"

    def open(self):
        self.fd = open(self.file_path, "w")
        self.first_record_written = False
        return self

    def close(self):
        self.fd.close()
        self.fd = None

    def write_record(self, obj):
        if self.first_record_written:
            self.fd.write(self.delimiter)
        self.fd.write(json.dumps(obj))
        self.first_record_written = True

    def __enter__(self):
        return self.open()

    def __exit__(self, type, value, traceback):
        self.close()

In [44]:
data_dir = os.path.join('data')
output_dir = os.path.join('output_data')

search_data_path = os.path.join(data_dir, 'torob-search-data_v1.jsonl')
aggregated_search_data_path = os.path.join(output_dir, 'aggregated_search_data.jsonl')

products_path = os.path.join(data_dir, 'products-info_v1.jsonl')
preprocessed_products_path = os.path.join(output_dir, 'preprocessed_products.jsonl')

test_data_path = os.path.join(data_dir, 'test-offline-data_v1.jsonl')
preprocessed_test_queries_path = os.path.join(output_dir, 'preprocessed_test_queries.jsonl')

In [45]:
def aggregate_searches(search_data_path, output_path):
    """Aggregate searches based on raw query.
    
    For each unique raw query in the search data, the frequency of products and
    clicked products would be aggregated.
    """
    agg_searches = defaultdict(
        lambda : dict(
            results=Counter(),
            clicks=Counter(),
        )
    )
    print("Aggregating searches based on raw query...")
    for search in tqdm(read_json_lines(search_data_path)):
        agg_searches[search['raw_query']]['results'].update(search['result'])
        agg_searches[search['raw_query']]['clicks'].update(search['clicked_result'])
    
    print('Writing aggregated searches into file...')
    with JSONLinesWriter(output_path) as out_file:
        for raw_query, stats in tqdm(agg_searches.items()):
            results, results_count = list(zip(*stats['results'].most_common()))
            clicks, clicks_count = list(zip(*stats['clicks'].most_common()))
            record = {
                'raw_query': raw_query,
                'raw_query_normalized': normalize_text(raw_query),
                'results': results,
                'results_count': results_count,
                'clicks': clicks,
                'clicks_count': clicks_count,
            }
            out_file.write_record(record)

    print("Finished aggregating searches.")
    print(f'Number of aggregate search records: {len(agg_searches)}')
    print(f"The aggregated searches data were stored in '{output_path}'.")

In [46]:
aggregate_searches(search_data_path, aggregated_search_data_path)

Aggregating searches based on raw query...


2499901it [01:14, 33356.38it/s]


Writing aggregated searches into file...


100%|██████████| 270099/270099 [00:12<00:00, 21476.56it/s]


Finished aggregating searches.
Number of aggregate search records: 270099
The aggregated searches data were stored in 'output_data\aggregated_search_data.jsonl'.


In [47]:
def preprocess_products(products_path, output_path):
    """Preprocess product names.
    
    The different titles of a product are concatenated together and 
    the resulting string would be normalized. Then, the normalized title
    is split into tokens and only the set of unique tokens would be selected
    as the final title of the product.
    """
    print('Preprocessing products...')
    count = 0
    with JSONLinesWriter(output_path) as out_file:
        for product in tqdm(read_json_lines(products_path)):
            titles = product['titles']
            titles_concat_normalized = normalize_text(" ".join(titles))
            titles_words_set = set(titles_concat_normalized.split())
            titles_words_concat = " ".join(titles_words_set)
            
            record = {
                'id': product['id'],
                'title_normalized': titles_words_concat,
            }
            out_file.write_record(record)
            count += 1
    print('Finished preprocessing products.')
    print(f'Number of processed products: {count}')
    print(f"The processed products data were stored in '{output_path}'")

In [48]:
preprocess_products(products_path, preprocessed_products_path)

Preprocessing products...


2546983it [01:13, 43579.55it/s]Downloading...
From (uriginal): https://drive.google.com/uc?id=1spSFY1yieMcjGJc989bbbEheNhegmEmR
From (redirected): https://drive.google.com/uc?id=1spSFY1yieMcjGJc989bbbEheNhegmEmR&confirm=t&uuid=b16c0d79-124d-40de-bff3-c7028c999592
To: c:\Users\Lenovo\Desktop\contest_data_v1\torob-data-challenge-2023_datafiles_v1.7z

  0%|          | 0.00/454M [00:00<?, ?B/s]
  0%|          | 524k/454M [00:00<07:39, 987kB/s]
  0%|          | 1.05M/454M [00:00<04:38, 1.63MB/s]
  0%|          | 1.57M/454M [00:00<03:28, 2.17MB/s]
  0%|          | 2.10M/454M [00:01<02:56, 2.56MB/s]
  1%|          | 2.62M/454M [00:01<02:37, 2.86MB/s]
  1%|          | 3.15M/454M [00:01<02:26, 3.08MB/s]
  1%|          | 3.67M/454M [00:01<02:12, 3.41MB/s]
  1%|          | 4.19M/454M [00:01<02:09, 3.46MB/s]
  1%|          | 4.72M/454M [00:01<02:06, 3.55MB/s]
  1%|          | 5.24M/454M [00:01<02:03, 3.65MB/s]
  1%|▏         | 5.77M/454M [00:01<02:03, 3.63MB/s]
  1%|▏         | 6.29M/454M [00:02<0

Finished preprocessing products.
Number of processed products: 3612277
The processed products data were stored in 'output_data\preprocessed_products.jsonl'


In [49]:
def preprocess_test_queries(test_data_path, output_path):
    """Normalize test queries."""
    print('Preprocessing test queries...')
    count = 0
    with JSONLinesWriter(output_path) as out_file:
        for test_sample in tqdm(read_json_lines(test_data_path)):
            normalized_query = normalize_text(test_sample['raw_query'])
            record = {
                'raw_query_normalized': normalized_query,
            }
            count += 1
            out_file.write_record(record)
    print('Finished preprocessing test queries.')
    print(f'Number of processed test queries: {count}')
    print(f"The processed test queries were stored in '{output_path}'")

In [50]:
preprocess_test_queries(test_data_path, preprocessed_test_queries_path)

Preprocessing test queries...


23140it [00:00, 61539.33it/s]

Finished preprocessing test queries.
Number of processed test queries: 23140
The processed test queries were stored in 'output_data\preprocessed_test_queries.jsonl'


## Feature extraction

In [28]:
# Reset environment due to memory constraints.
%reset -f

In [29]:
import os
import json
import gc
import pickle

from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
def read_json_lines(path, n_lines=None):
    """Creates a generator which reads and returns lines of
    a json lines file, one line at a time, each as a dictionary.
    
    This could be used as a memory-efficient alternative of `pandas.read_json`
    for reading a json lines file.
    """
    with open(path, 'r') as f:
        for i, line in enumerate(f):
            if n_lines == i:
                break
            yield json.loads(line)

In [31]:
output_dir = os.path.join('output_data')

aggregated_search_data_path = os.path.join(output_dir, 'aggregated_search_data.jsonl')
preprocessed_products_path = os.path.join(output_dir, 'preprocessed_products.jsonl')
preprocessed_test_queries_path = os.path.join(output_dir, 'preprocessed_test_queries.jsonl')

train_dat_file_path = os.path.join(output_dir, 'train.dat')

random_projection_mat_path = os.path.join(output_dir, 'random_projection_mat.npy')
product_features_path = os.path.join(output_dir, 'product_features.npy')
queries_train_features_path = os.path.join(output_dir, 'queries_train_features.npy')
queries_test_features_path = os.path.join(output_dir, 'queries_test_features.npy')
products_id_to_idx_path = os.path.join(output_dir, 'products_id_to_idx.pkl')

In [34]:
# Number of tokens in the vocabulary of TF-IDF.
VOCAB_SIZE = 4096
# Embedding dimension used for random projection of TF-IDF vectors.
EMBEDDING_DIM = 256
# Number of training samples to use (set to None to use all samples).
NUM_TRAIN_SAMPLES = 50_000

In [35]:
# Load aggregated search data which will be used as training data.
aggregated_searches_df = pd.DataFrame(
    read_json_lines(aggregated_search_data_path, n_lines=NUM_TRAIN_SAMPLES)
)

In [36]:
aggregated_searches_df

,raw_query,raw_query_normalized,results,results_count,clicks,clicks_count
0,لوستر سقفی برنز,لوستر سقفی برنز,"[6462477, 7385791, 3775536, 1663988, 6946932, ...","[22, 22, 21, 20, 20, 20, 20, 20, 19, 19, 19, 1...","[7151290, 2819798, 6462477, 6254687, 9930140, ...","[4, 3, 3, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, ..."
1,قیمت هلیکوپتر,قیمت هلیکوپتر,"[363737, 3147253, 9796388, 1420685, 5680879, 1...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, ...","[3147253, 7135387, 9796388, 8630956]","[1, 1, 1, 1]"
2,ساعت هوشمند,ساعت هوشمند,"[None, 9391819, 8563833, 2459592, 7824893, 990...","[9654, 9189, 9117, 9086, 9085, 9081, 9069, 903...","[9391819, 4229448, 2459592, 8563833, 3200289, ...","[2559, 2211, 2094, 1111, 1046, 851, 629, 519, ..."
3,تفلون مایع,تفلون مایع,"[None, 5428407, 4474271, 4581189, 8504749, 413...","[12, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,...","[4131340, 7660686, 5901997, 2376830, 2383125, ...","[2, 1, 1, 1, 1, 1, 1]"
4,خط زن,خط زن,"[1105044, 9285461, 1791930, 7624894, 8466568, ...","[498, 495, 495, 490, 489, 489, 488, 487, 472, ...","[8466568, 9285461, 1314054, 1105044, 9736119, ...","[181, 114, 91, 57, 57, 56, 45, 44, 39, 36, 35,..."
...,...,...,...,...,...,...
49995,کانن 3010,کانن 3010,"[5775551, 3826318, 2336292, 947939, 5803582, 5...","[6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 4, 4, 4, ...","[5775551, 2327611, None]","[4, 3, 1]"
49996,mi11tpro,mi11tpro,"[7437752, 3211023, 743379, 1898330, 2282190, 2...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, ...","[7437752, 3211023, 1898330]","[3, 3, 1]"
49997,گوشیهای,گوشیهای,"[9446310, 7698457, 483913, 6133410, 707374, 35...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2334880, 3968010, 408480]","[2, 1, 1]"
49998,جعبه بکس ریز,جعبه بکس ریز,"[None, 9432861, 1652719, 936541, 9069771, 9069...","[34, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5,...","[None, 5161950, 4645051, 6763200, 6731340, 848...","[7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"


In [37]:
# Load preprocessed product data.
products_data_df = pd.DataFrame(read_json_lines(preprocessed_products_path))

In [38]:
# Load preprocessed test queries.
test_offline_queries_df = pd.DataFrame(read_json_lines(preprocessed_test_queries_path))

In [39]:
# Create a mapping from ID of products to their integer index.
products_id_to_idx = dict(
    (p_id, idx)
    for idx, p_id in enumerate(products_data_df['id'])
)

In [40]:
# Create a random matrix which will be used for projection of
# TF-IDF vector to a lower-ranked random space.
random_projection_mat = np.random.rand(VOCAB_SIZE, EMBEDDING_DIM)

vectorizer = TfidfVectorizer(max_features=VOCAB_SIZE, lowercase=True, use_idf=True)

# Fit tf-idf vectorizer on normalized product names and compute their tf-idf vectors.
products_tfidf = vectorizer.fit_transform(products_data_df['title_normalized'])
# Project the tf-idf vectors using random projection matrix.
products_projected = products_tfidf.dot(random_projection_mat)
del products_tfidf  # Free up memory.
gc.collect()

# Transform the training raw queries into tf-idf vectors.
queries_train_tfidf = vectorizer.transform(aggregated_searches_df['raw_query_normalized'])
queries_train_projected = queries_train_tfidf.dot(random_projection_mat)
del queries_train_tfidf # Free up memory.
gc.collect();

In [41]:
# Transform test raw queries into tf-idf vectors.
queries_test_tfidf = vectorizer.transform(test_offline_queries_df['raw_query_normalized'])
queries_test_projected = queries_test_tfidf.dot(random_projection_mat)
del queries_test_tfidf # Free up memory.
gc.collect();

In [42]:
del vectorizer
gc.collect();

For each pair of (Q, P) where Q is a search query and P is a product in the search-results of query Q, a training sample is generated. The features of Q and P will be their corresponding random-projected TF-IDF vectors. Further, the sample is labeled based on the number of clicks on P (in the search results of query Q).

In [44]:
def create_dat_file(
    dat_file_path,
    agg_searches_df,
    query_features,
    product_features,
    n_candidates=None,
):
    """
    Create a `dat` file which is the training data of LambdaMart model.

    The file format of the training and test files is the same as for SVMlight,
    with the exception that the lines in the input files have to be sorted by increasing qid.
    The first lines may contain comments and are ignored if they start with #.
    Each of the following lines represents one training example and is of the following format:

    <line> .=. <target> qid:<qid> <feature>:<value> <feature>:<value> ... <feature>:<value> # <info>
    <target> .=. <float>
    <qid> .=. <positive integer>
    <feature> .=. <positive integer>
    <value> .=. <float>
    <info> .=. <string>

    The target value and each of the feature/value pairs are separated by a space character.
    Feature/value pairs MUST be ordered by increasing feature number.
    Features with value zero can be skipped.
    The target value defines the order of the examples for each query.
    Implicitly, the target values are used to generated pairwise preference constraints as described in [Joachims, 2002c].
    A preference constraint is included for all pairs of examples in the example_file, for which the target value differs.
    The special feature "qid" can be used to restrict the generation of constraints.
    Two examples are considered for a pairwise preference constraint only if the value of "qid" is the same.

    For example, given the example_file

    3 qid:1 1:1 2:1 3:0 4:0.2 5:0 # 1A
    2 qid:1 1:0 2:0 3:1 4:0.1 5:1 # 1B
    1 qid:1 1:0 2:1 3:0 4:0.4 5:0 # 1C
    1 qid:1 1:0 2:0 3:1 4:0.3 5:0 # 1D
    1 qid:2 1:0 2:0 3:1 4:0.2 5:0 # 2A
    2 qid:2 1:1 2:0 3:1 4:0.4 5:0 # 2B
    1 qid:2 1:0 2:0 3:1 4:0.1 5:0 # 2C
    1 qid:2 1:0 2:0 3:1 4:0.2 5:0 # 2D
    2 qid:3 1:0 2:0 3:1 4:0.1 5:1 # 3A
    3 qid:3 1:1 2:1 3:0 4:0.3 5:0 # 3B
    4 qid:3 1:1 2:0 3:0 4:0.4 5:1 # 3C
    1 qid:3 1:0 2:1 3:1 4:0.5 5:0 # 3D

    the following set of pairwise constraints is generated (examples are referred to by the info-string after the # character):

    1A>1B, 1A>1C, 1A>1D, 1B>1C, 1B>1D, 2B>2A, 2B>2C, 2B>2D, 3C>3A, 3C>3B, 3C>3D, 3B>3A, 3B>3D, 3A>3D

    More information:
     - https://xgboost.readthedocs.io/en/latest/tutorials/input_format.html#embedding-additional-information-inside-libsvm-file
     - https://www.cs.cornell.edu/people/tj/svm_light/svm_rank.html
    """
    with open(dat_file_path, "w") as file:
        for qid, agg_search in tqdm(enumerate(agg_searches_df.itertuples(index=False))):
            if n_candidates is None:
                limit = len(agg_search.results)
            else:
                limit = min(n_candidates, len(agg_search.results))
            clicks = dict(zip(agg_search.clicks, agg_search.clicks_count))

            for candidate_product_id in agg_search.results[:limit]:
                if candidate_product_id is None:
                    continue
                candidate_score = clicks.get(candidate_product_id, 0)
                candidate_score = np.log2(candidate_score + 1)

                p_idx = products_id_to_idx[candidate_product_id]
                features = np.concatenate((product_features[p_idx], query_features[qid]))
                features = np.around(features, 3)

                file.write(
                    f"{candidate_score} qid:{qid} "
                    + " ".join([f"{i}:{s}" for i, s in enumerate(features)])
                    + "\n"
                )

In [45]:
create_dat_file(
    train_dat_file_path,
    aggregated_searches_df,
    queries_train_projected,
    products_projected,
    n_candidates=200,
)

33303it [27:53, 35.23it/s]

In [17]:
# Since memory is limited, we store all the neccessary data
# such as extracted features on disk. Later, in inference
# step we may need some of these files.
np.save(random_projection_mat_path, random_projection_mat)
np.save(product_features_path, products_projected)
np.save(queries_train_features_path, queries_train_projected)
np.save(queries_test_features_path, queries_test_projected)

In [18]:
with open(products_id_to_idx_path, 'wb') as f:
    pickle.dump(products_id_to_idx, f)

## Training of model

In [21]:
# Reset due to memory constraints.
%reset -f

In [22]:
import os
import xgboost as xgb

In [23]:
output_dir = os.path.join('output_data')

train_dat_path = os.path.join(output_dir, 'train.dat')
model_path = os.path.join(output_dir, 'ranker.json')

In [24]:
train_data = xgb.DMatrix(train_dat_path)

XGBoostError: bad allocation

In [25]:
param = {
    "max_depth": 20,
    "eta": 0.3,
    "objective": "rank:ndcg",
    "verbosity": 1,
    "num_parallel_tree": 1,
    "tree_method": "gpu_hist",
    "eval_metric": ["ndcg"],
}
eval_list = [(train_data, "train")]

model = xgb.train(
    param,
    train_data,
    num_boost_round=200,
    evals=eval_list,
)

model.save_model(model_path)

NameError: name 'train_data' is not defined

## Inference (prediction) on test data

In [ ]:
# Reset due to memory constraints.
%reset -f

In [ ]:
import os
import pickle
import json

from tqdm import tqdm
import pandas as pd
import numpy as np
import xgboost as xgb

In [ ]:
def read_json_lines(path, n_lines=None):
    """Creates a generator which reads and returns lines of
    a json lines file, one line at a time, each as a dictionary.
    
    This could be used as a memory-efficient alternative of `pandas.read_json`
    for reading a json lines file.
    """
    with open(path, 'r') as f:
        for i, line in enumerate(f):
            if n_lines == i:
                break
            yield json.loads(line)

In [ ]:
data_dir = os.path.join('data')
output_dir = os.path.join('output_data')

test_data_path = os.path.join(data_dir, 'test-offline-data_v1.jsonl')

product_features_path = os.path.join(output_dir, 'product_features.npy')
queries_test_features_path = os.path.join(output_dir, 'queries_test_features.npy')
products_id_to_idx_path = os.path.join(output_dir, 'products_id_to_idx.pkl')

predictions_path = os.path.join(output_dir, 'predictions.txt')

model_path = os.path.join(output_dir, 'ranker.json')

In [ ]:
# Load projected products and queries data.
products_projected = np.load(product_features_path)
queries_test_projected = np.load(queries_test_features_path)
with open(products_id_to_idx_path, 'rb') as f:
    products_id_to_idx = pickle.load(f)

In [ ]:
# Load original test data which contains the result to be ranked.
test_data_df = pd.DataFrame(read_json_lines(test_data_path))

In [ ]:
# Load trained LambdaMART model.
param = {}
model = xgb.Booster(**param)
model.load_model(model_path)

In [ ]:
BATCH_SIZE = 64
test_predictions = []
for batch_idx in tqdm(range(0, len(test_data_df), BATCH_SIZE)):
    batch_data = test_data_df['result_not_ranked'].iloc[batch_idx:batch_idx + BATCH_SIZE]
    batch_features = []
    for test_qid, test_candidates in enumerate(batch_data, start=batch_idx):
        test_query_projected = queries_test_projected[test_qid]
        for candidate_pid in test_candidates:
            p_idx = products_id_to_idx[candidate_pid]
            features = np.concatenate((products_projected[p_idx], test_query_projected))
            batch_features.append(features)
    
    batch_features = np.stack(batch_features)
    batch_features = xgb.DMatrix(batch_features)
    batch_preds = model.predict(batch_features)
    
    start_idx = 0
    for test_candidates in batch_data:
        preds_sample = batch_preds[start_idx:start_idx + len(test_candidates)]
        sorted_idx = np.argsort(preds_sample)[::-1]
        sorted_candidates = [test_candidates[i] for i in sorted_idx]
        test_predictions.append(sorted_candidates)
        start_idx += len(test_candidates)

In [ ]:
def write_test_predictions(predictions_path, predictions):
    lines = []
    for preds in predictions:
        lines.append(",".join([str(p_id) for p_id in preds]))

    with open(predictions_path, 'w') as f:
        f.write("\n".join(lines))

In [ ]:
write_test_predictions(predictions_path, test_predictions)